In [1]:
# import packages

import cv2
import numpy as np
import tensorflow as tf
import mediapipe as mp

from tensorflow.keras.models import load_model
from os import (
    listdir,
    path,
    makedirs
)

In [2]:
def main(photos):

    # Initialize mediapipe utilities
    mpHands = mp.solutions.hands
    hands = mpHands.Hands(max_num_hands=1, min_detection_confidence=0.7)
    mpDraw = mp.solutions.drawing_utils

    # Load the gesture recognition model
    model = load_model('mp_hand_gesture')

    # Add variables to store last position of landmarks
    last_finger_1_position = None
    last_finger_2_position = None

    # Photo index counter
    i = 0

    # Load class names
    f = open('gesture.names', 'r')
    classNames = f.read().split('\n')
    f.close()

    # Initialize the webcam
    cap = cv2.VideoCapture(0)

    # Sets sensibility of hand movements
    move_range = 30

    while True:
        # Read tge frame
        _, frame = cap.read()

        x, y, c = frame.shape

        # Flip the frame vertically
        frame = cv2.flip(frame, 1)
        framergb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Get hand landmark prediction
        result = hands.process(framergb)

        className = ''

        final_pic = frame

        # Post-processing the result if hands are detected in the picture
        if result.multi_hand_landmarks and len(photos) != 0:
            landmarks = []

            for handslms in result.multi_hand_landmarks:
                # Appending coordinates of landmarks to the landmarks 
                for lm in handslms.landmark:
                    lmx = int(lm.x * x)
                    lmy = int(lm.y * y)

                    landmarks.append([lmx, lmy])

                # Drawing landmarks on frames
                mpDraw.draw_landmarks(frame, handslms, mpHands.HAND_CONNECTIONS)

                # Predict gesture
                prediction = model.predict([landmarks])

                # Set className
                classID = np.argmax(prediction)
                className = classNames[classID]

            # Check hand direction changes based on last position   
            if last_finger_1_position is None or last_finger_2_position is None:
                pass
            elif landmarks[8][0] > last_finger_1_position + move_range and landmarks[12][0] > last_finger_2_position + move_range and i != len(photos) - 1:
                i += 1  
            elif landmarks[8][0] < last_finger_1_position - move_range and landmarks[12][0] < last_finger_2_position - move_range and i!= 0:
                i -= 1
            else:
                pass    

            last_finger_1_position = landmarks[8][0]
            last_finger_2_position = landmarks[12][0]

            # Write habd gesture class name on the frame 
            cv2.putText(frame, className, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3, cv2.LINE_AA)

            height, width, color = frame.shape

            # Read and resize the picture
            pic = cv2.imread(f"photos\\{photos[i]}")
            pic = cv2.resize(pic, (640, 480))

            final_pic = pic

            # Opens two windows, one for frame and one for pitcure
            cv2.imshow('final_frame', final_pic)
            cv2.imshow('frame', frame)

        cv2.imshow('final_frame', final_pic)
        cv2.imshow('frame', frame)

        if cv2.waitKey(1) == ord('q'):
            break

    # release the webcam and destroy all active windows
    cap.release()
    cv2.destroyAllWindows()

In [9]:
if not path.exists("photos\\"):
  print(f'Folder "photos" does not exist, creating...')
  makedirs("photos")
  print("Folder created, add photos and rerun this cell to navigate")

else:
  print(f'Folder "photos" already exists')

# Get list of photos in directory
photos = listdir('photos\\')

if len(photos) == 0:
    print('Folder "photos" is empty, add photos and rerun this cell to navigate')
else:
  main(photos)  

Folder "photos" already exists
1/1 [==============================] - 0s 28ms/step
